In [622]:
import bitarray
import math
import sys

import numpy as np
import scipy.io.wavfile as wav

from IPython.display import Audio

np.set_printoptions(threshold=sys.maxsize)

In [623]:
def phase_enc(signal, text, L=1024):
  plain = signal
  data = bitarray.bitarray()
  data.frombytes(text.encode('ascii'))
  I = len(plain)
  m = len(data)
  N = math.floor(I / L)

  s = np.reshape(plain[: N * L], (L, N))

  w = np.fft.fft(s, axis=0)
  Phi = np.angle(w)
  A = np.abs(w)

  DeltaPhi = np.zeros((L, N))
  for k in range(1, N):
    DeltaPhi[:, k] = Phi[:, k] - Phi[:, k - 1]

  PhiData = np.zeros(m)
  for k in range(m):
    if data[k] == 0:
      PhiData[k] = np.pi / 2
    else:
      PhiData[k] = -np.pi / 2

  Phi_new = Phi.copy()
  Phi_new[L // 2 - m : L // 2, 0] = PhiData
  Phi_new[L // 2 : L // 2 + m, 0] = -np.flip(PhiData)

  for k in range(1, N):
      Phi_new[:, k] = Phi_new[:, k - 1] + DeltaPhi[:, k]

  z = np.real(np.fft.ifft(np.multiply(A, np.exp(1j * Phi_new)), axis=0))
  snew = np.reshape(z, (N * L))
  out = np.concatenate((snew, plain[N * L:]))

  return np.asarray(out, dtype=np.int16)


In [624]:
fs, x = wav.read('sample.wav')
print(f'Sample rate is {fs} Gz')
print(f'File contains {x.shape[0]} samples')
print(f'Approximate audio duration is {x.shape[0] / fs} s')
Audio('sample.wav')

Sample rate is 44100 Gz
File contains 140928 samples
Approximate audio duration is 3.1956462585034013 s


In [625]:
msg = 'h'
xmod = phase_enc(x[:, 0], msg)

wav.write('sample-mod.wav', fs, xmod)
Audio('sample-mod.wav')

bitarray('01101000')


In [626]:
def phase_dec(signal, L_msg, L=1024):
    plain = signal
    I = len(plain)
    N = math.floor(I / L)
    s = np.reshape(plain[: N * L], (L, N))
    w = np.fft.fft(s, axis=0)
    Phi = np.angle(w)
    Phi = Phi[:L, 0]

    m = 8 * L_msg  # Length of bit sequence (for 8 bit)

    # Retrieving data back from phases of first segments
    data = ''
    for k in range(m):
        angle = Phi[L//2 - m + k]
        if angle > 0:
            data += '0'
        else:
            data += '1'

    print(data)

    # Converting binary data to ASCII characters
    bin_str = [data[i:i+8] for i in range(0, m, 8)]
    out = ''.join([chr(int(b, 2)) for b in bin_str])

    return out

In [627]:
fs, x = wav.read('sample-mod.wav')
phase_dec(x, len(msg))

00111100


'<'